# URBANopt to Alfalfa workflow 

In [ ]:
import os
import shutil
import json
import pandas
import datetime
import time
from alfalfa_client.alfalfa_client import AlfalfaClient
from pprint import pprint
from pathlib import Path

### Define alfalfa client object

Use `pip install alfalfa-client` prior to running further commands

In [ ]:
ac = AlfalfaClient(host='https://cctwin.nrel.gov')

### Define Alfalfa simulation parameters

In [ ]:
# If you are using historian, you will need to search for this time period in Grafana dashboard to view results.
start_dt = datetime.datetime(2021, 7, 1, 12, 2, 0)
end_dt = datetime.datetime(2021, 7, 3, 0, 0, 0)

# For external_clock == true, API calls are used to advance the model.  
# If external_clock == false, Alfalfa will handle advancing the model according to a specified timescale (timescale 1 => realtime)
params = {
    "external_clock": True,
    "start_datetime": start_dt,
    "end_datetime": end_dt
}

### Create folders to upload to Alfalfa from the URBANopt Project

The code below will create folders for each URBANopt building model that can be uploaded to Alfalfa. The folder contains: 

    - Model Folder: Contains OpenStudio model (.osm file) for each building created using the URBANopt simulation.
    - Measures Folder: Measures to be added while running Alfalfa in the OpenStudio Workflow
    - Weather Folder: Contains EPW weather file 
    - workflow.osw file: OpenStudio workflow file

**Define the following variables before running the code**:

- `uo_folder` : URBANopt Scenario Directory 
- `weather`: Define weather file used in URBANopt project
- `workflow`: OpenStudio workflow file name

In [ ]:
# Set URBANopt scenario directory
uo_folder = Path('C:/a/baseline_stochastic/run/baseline_scenario_stochastic')

uo_scenario_folder = uo_folder.name
uo_scenario_path = Path(f"./{uo_scenario_folder}")
uo_scenario_path.mkdir(parents=True, exist_ok=True)

print(f"Folders saved at {uo_scenario_path}")

# Add .epw filename
weather = "USA_CO_Denver.Intl.AP.725650_TMY3.epw"

# Add .osw filename
workflow = "workflow.osw"

for file in uo_folder.iterdir():
        
    if file.is_dir() and (file / 'in.osm').exists():
        uo_model_filepath = uo_scenario_path / file.name
        uo_model_filepath.mkdir(parents=True, exist_ok=True)
        (uo_model_filepath / 'models').mkdir(parents=True, exist_ok=True)
        (uo_model_filepath / 'measures').mkdir(parents=True, exist_ok=True)
        (uo_model_filepath / 'weather').mkdir(parents=True, exist_ok=True)

        shutil.copy((uo_folder / file / 'in.osm'), (uo_model_filepath / 'models' / f'{file}.osm'))
        shutil.copy((uo_folder / '../../weather' / weather), (uo_model_filepath / 'weather'))

        #shutil.copy((os.path.join(uo_folder,filename, 'in.osm')), (os.path.join(uo_model_filepath, 'models', '{}.osm'.format(filename))))
        #shutil.copy((os.path.join(uo_folder, '../../weather', weather)), (os.path.join(uo_model_filepath, 'weather')))
    
        osw = {"seed_file": f"{file}.osm",
           "weather_file": f"{weather}",
           "measure_paths": "./measures",
           "run_directory": "./run/",
           "file_paths": [
               "./weather/",
               "./models/"
           ]
          }
        
        f = open((uo_model_filepath / workflow), "w+")
        f.write(json.dumps(osw, indent=4))
        f.close()

print('Done')       

### Upload models to Alfalfa

- Submit models to Alfalfa server
- Print model inputs (EP Actuators)

In [ ]:
for file in uo_scenario_path.iterdir():
    
    model_id = ac.submit(file)
    print(model_id)
        
    # Start Simulations
    print(f"Starting site: {model_id}")
    ac.start(model_id, **params)
        
    # Get model input points
    print(f"{model_id} inputs:")
    pprint(ac.get_inputs(model_id)) 
       

### Set model input point

To set an input value use the `ac.set_inputs(site_id, inputs)` function.

- `site_id` - the id of the site returned by the `ac.submit` function
- `inputs_dict` - a dictionary of input names and the desired values

In [ ]:
input_dict = {'Core_ZN_ZN_PSZ_AC_1_Outside_Air_Damper_CMD': 0.7}
ac.set_inputs(model_id, input_dict)

### Advance the model
12/10/2021: timestep is hardcoded to 1 minute w/in Alfalfa worker.

In [ ]:
timesteps = 5
for _ in range(timesteps):
    ac.advance(model_id)
    print(f"Model advanced to time: {ac.get_sim_time(model_id)}")

### Get model's outputs
Query the outputs (EP Sensors) of the models as well as their values

In [ ]:
print(f"{model_id} outputs:")
pprint(ac.outputs(model_id))

### Stop the simulations

In [ ]:
ac.stop(model_id)